In [1]:
#importing nesessary libraries
import pandas as pd
import matplotlib as plt
import numpy as np

In [2]:
#Loading label data from csv files to dataframe
df_label = pd.read_csv('DataWrangling_prod-by-operated-day.csv')
df_label=df_label[['api','cum_oil_365']].dropna()
df_label
#df_label.info()
#len(df_label['api'].unique()) #making sure api numbers are not repeted

,api,cum_oil_365
0,33007001030000,7992.0
4,33007011110000,1972.0
5,33007011160000,1549.0
6,33007011480000,1362.0
7,33007012030000,35061.0
...,...,...
8630,33105031490000,49423.0
8640,33105031600000,95125.0
8641,33105031610000,53310.0
8642,33105031620000,89055.0


In [3]:
#Loading feature data from csv file to dataframe
df_compl = pd.read_csv('completion.csv',parse_dates=['treatment_date']).sort_values('api') 
df_compl.drop(['file_number'],axis=1, inplace=True) 
df_compl['number_treatments']=1 #number of treatments performed for a well
df_compl['number_intervals']=1 #
df_compl['interval'] = df_compl['bottom']-df_compl['top'] #thickness of the producing interval
df_compl.info()
len(df_label['api'].unique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9664 entries, 9126 to 7830
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   api                9664 non-null   int64         
 1   bottom             9664 non-null   int64         
 2   fluid_bbl          9664 non-null   int64         
 3   fluid_bbl_per_ft   9664 non-null   int64         
 4   fluid_gal          9664 non-null   int64         
 5   fluid_gal_per_ft   9664 non-null   int64         
 6   ft_per_stage       9664 non-null   int64         
 7   lateral_length     9664 non-null   int64         
 8   max_treat_press    9664 non-null   int64         
 9   max_treat_rate     9664 non-null   float64       
 10  propp_lbs          9664 non-null   int64         
 11  propp_lbs_per_ft   9664 non-null   int64         
 12  stages             9664 non-null   int64         
 13  top                9664 non-null   int64         
 14  treat

6307

In [4]:
#Loading feature data from csv file to dataframe and making types correct
df_wells = pd.read_csv('well-index.csv', parse_dates=['Well_Status_Date','Spud_Date']).sort_values('api')
df_categories=['Current_Operator','Current_Well_Name','Well_Type', 'Well_Status', 
               'Wellbore','Lease_Name', 'Lease_Number','Original_Operator',
              'Original_Well_Name', 'County_Name', 'Township','Range','QQ',
              'Footages','Field_Name','Produced_Pools','Wellbore']
df_wells[df_categories]=df_wells[df_categories].astype('category')
df_wells['Well_Status_Date'].replace('Confidential', '', inplace=True)
df_wells['Well_Status_Date'] = df_wells['Well_Status_Date'].astype('datetime64')
df_wells['Wellbore']=df_wells['Wellbore'].astype('category')
df_wells.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10687 entries, 2 to 10683
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   api                 10687 non-null  int64         
 1   file_no             10687 non-null  int64         
 2   Current_Operator    10687 non-null  category      
 3   Current_Well_Name   10687 non-null  category      
 4   Lease_Name          10687 non-null  category      
 5   Lease_Number        10682 non-null  category      
 6   Original_Operator   10687 non-null  category      
 7   Original_Well_Name  10686 non-null  category      
 8   Spud_Date           10687 non-null  datetime64[ns]
 9   TD                  9124 non-null   float64       
 10  County_Name         10687 non-null  category      
 11  Township            10687 non-null  category      
 12  Range               10687 non-null  category      
 13  Section             10687 non-null  int64     

In [5]:
#merging dataframes into one
df_comb = pd.merge(df_label,df_wells,on=['api'])
df_comb

,api,cum_oil_365,file_no,Current_Operator,Current_Well_Name,Lease_Name,Lease_Number,Original_Operator,Original_Well_Name,Spud_Date,...,Footages,Field_Name,Produced_Pools,Wellbore,Latitude,Longitude,Well_Type,Well_Status,CTB,Well_Status_Date
0,33007015830000,28758.0,16431,WHITING OIL AND GAS CORPORATION,BSMU 1305,BSMU,1305,WHITING OIL AND GAS CORPORATION,BSMU 1305,2010-04-10,...,906 FSL 1197 FEL,BIG STICK,MADISON,DIRECTIONAL,47.112737,-103.418030,OG,A,406270.0,2010-05-26
1,33007016320000,19649.0,17768,XTO ENERGY INC.,CHRUSZCH 43X-29F,CHRUSZCH,43X-29F,XTO ENERGY INC.,CHRUSZCH 43X-29,2012-04-21,...,2400 FSL 300 FEL,ST. DEMETRIUS,BAKKEN,HORIZONTAL,47.088015,-103.244706,OG,A,117768.0,2012-07-25
2,33007016380000,39719.0,18149,XTO ENERGY INC.,FEDERAL 13-30R,FEDERAL,13-30R,"ENCORE OPERATING, L.P.",FEDERAL 13-30R,2009-06-16,...,1232 FSL 621 FWL,MAGPIE,DUPEROW,DIRECTIONAL,47.258726,-103.411020,OG,A,118149.0,2009-07-29
3,33007016390000,10203.0,18216,"WILLISTON EXPLORATION, LLC",VANVIG 1,VANVIG,1,"DAKOTA MINERALS, LLC",VANVIG 1,2011-07-27,...,2100 FSL 1980 FEL,WILDCAT,TYLER,VERTICAL,46.736696,-103.553665,OG,A,118216.0,2011-10-23
4,33007016410000,33166.0,18502,WHITING OIL AND GAS CORPORATION,TEDDY 44-13TFH,TEDDY,44-13TFH,WHITING OIL AND GAS CORPORATION,TEDDY 44-13TFH,2010-07-25,...,505 FSL 541 FEL,BIG STICK,BAKKEN,HORIZONTAL,47.111642,-103.415385,OG,A,118502.0,2010-11-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5908,33105031490000,49423.0,26084,"HRC OPERATING, LLC",QUARNE 2-21-16H,QUARNE,2-21-16H,"HRC OPERATING, LLC",QUARNE 2-21-16H,2013-09-17,...,325 FSL 2100 FEL,STRANDAHL,BAKKEN,HORIZONTAL,48.402825,-103.871552,OG,A,220844.0,2013-11-05
5909,33105031600000,95125.0,26123,"CONTINENTAL RESOURCES, INC.",MORGAN 1-29H1,MORGAN,1-29H1,"CONTINENTAL RESOURCES, INC.",MORGAN 1-29H1,2013-08-28,...,395 FNL 2640 FWL,CORINTH,BAKKEN,HORIZONTAL,48.574285,-103.243790,OG,A,126123.0,2013-11-12
5910,33105031610000,53310.0,26133,OASIS PETROLEUM NORTH AMERICA LLC,BURLESON 5502 41-7B,BURLESON,5502 41-7B,OASIS PETROLEUM NORTH AMERICA LLC,BURLESON 5502 41-7B,2013-09-26,...,325 FSL 454 FWL,SQUIRES,BAKKEN,HORIZONTAL,48.256680,-103.860935,OG,A,218801.0,2013-11-08
5911,33105031620000,89055.0,26134,"PETRO-HUNT, L.L.C.",BOSS 154-99-17A-18-6H,BOSS,154-99-17A-18-6H,"PETRO-HUNT, L.L.C.",BOSS 154-99-17A-18-6H,2013-10-11,...,300 FNL 250 FEL,STOCKYARD CREEK,BAKKEN,HORIZONTAL,48.168127,-103.432325,OG,A,126134.0,2013-12-27


In [6]:
#merging dataframes witth labels, well info, and completions into one on unique well id (API) key
df_raw = pd.merge(df_comb,df_compl,on=['api']).sort_values('api')
df_raw

,api,cum_oil_365,file_no,Current_Operator,Current_Well_Name,Lease_Name,Lease_Number,Original_Operator,Original_Well_Name,Spud_Date,...,max_treat_press,max_treat_rate,propp_lbs,propp_lbs_per_ft,stages,top,treatment_date,number_treatments,number_intervals,interval
0,33007015830000,28758.0,16431,WHITING OIL AND GAS CORPORATION,BSMU 1305,BSMU,1305,WHITING OIL AND GAS CORPORATION,BSMU 1305,2010-04-10,...,0,0.0,0,0,1,9398,2010-05-12,1,1,43
1,33007016320000,19649.0,17768,XTO ENERGY INC.,CHRUSZCH 43X-29F,CHRUSZCH,43X-29F,XTO ENERGY INC.,CHRUSZCH 43X-29,2012-04-21,...,8461,48.0,2757929,287,30,11224,2012-07-19,1,1,9586
2,33007016380000,39719.0,18149,XTO ENERGY INC.,FEDERAL 13-30R,FEDERAL,13-30R,"ENCORE OPERATING, L.P.",FEDERAL 13-30R,2009-06-16,...,0,0.0,0,0,1,11370,2010-11-08,1,1,6
3,33007016380000,39719.0,18149,XTO ENERGY INC.,FEDERAL 13-30R,FEDERAL,13-30R,"ENCORE OPERATING, L.P.",FEDERAL 13-30R,2009-06-16,...,0,0.0,0,0,1,11370,2009-07-29,1,1,6
4,33007016390000,10203.0,18216,"WILLISTON EXPLORATION, LLC",VANVIG 1,VANVIG,1,"DAKOTA MINERALS, LLC",VANVIG 1,2011-07-27,...,0,0.0,0,0,1,7938,2011-08-10,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6286,33105031490000,49423.0,26084,"HRC OPERATING, LLC",QUARNE 2-21-16H,QUARNE,2-21-16H,"HRC OPERATING, LLC",QUARNE 2-21-16H,2013-09-17,...,9440,78.4,4056680,417,35,10527,2013-11-01,1,1,9727
6287,33105031600000,95125.0,26123,"CONTINENTAL RESOURCES, INC.",MORGAN 1-29H1,MORGAN,1-29H1,"CONTINENTAL RESOURCES, INC.",MORGAN 1-29H1,2013-08-28,...,7120,41.0,3763960,403,30,10165,2013-10-20,1,1,9330
6288,33105031610000,53310.0,26133,OASIS PETROLEUM NORTH AMERICA LLC,BURLESON 5502 41-7B,BURLESON,5502 41-7B,OASIS PETROLEUM NORTH AMERICA LLC,BURLESON 5502 41-7B,2013-09-26,...,7447,25.7,3527746,370,32,10892,2013-10-31,1,1,9533
6289,33105031620000,89055.0,26134,"PETRO-HUNT, L.L.C.",BOSS 154-99-17A-18-6H,BOSS,154-99-17A-18-6H,"PETRO-HUNT, L.L.C.",BOSS 154-99-17A-18-6H,2013-10-11,...,9366,38.6,3667160,403,33,11603,2013-11-06,1,1,9093


In [7]:
#checking column types
df_raw.dtypes

api                            int64
cum_oil_365                  float64
file_no                        int64
Current_Operator            category
Current_Well_Name           category
Lease_Name                  category
Lease_Number                category
Original_Operator           category
Original_Well_Name          category
Spud_Date             datetime64[ns]
TD                           float64
County_Name                 category
Township                    category
Range                       category
Section                        int64
QQ                          category
Footages                    category
Field_Name                  category
Produced_Pools              category
Wellbore                    category
Latitude                     float64
Longitude                    float64
Well_Type                   category
Well_Status                 category
CTB                          float64
Well_Status_Date      datetime64[ns]
bottom                         int64
f

In [8]:
#checking for unique well ids
len(df_raw['api'].unique())

5848

In [9]:
# Removing duplicate rows
df_nd=df_raw[np.append(df_compl.columns.values,'Produced_Pools')]
for i in df_nd.columns:
    duplicateRowsDF = df_nd[df_nd.duplicated(df_nd.columns.drop(i),keep='first')]
    if len(duplicateRowsDF)>0:
        df_nd=df_nd.drop(duplicateRowsDF.index)
        print(i)
        #print(i, ' ', duplicateRowsDF, sep='\n')
df_nd

api
treatment_date


,api,bottom,fluid_bbl,fluid_bbl_per_ft,fluid_gal,fluid_gal_per_ft,ft_per_stage,lateral_length,max_treat_press,max_treat_rate,propp_lbs,propp_lbs_per_ft,stages,top,treatment_date,number_treatments,number_intervals,interval,Produced_Pools
0,33007015830000,9441,47,1,1998,46,43,43,0,0.0,0,0,1,9398,2010-05-12,1,1,43,MADISON
1,33007016320000,20810,57661,6,2421762,252,319,9586,8461,48.0,2757929,287,30,11224,2012-07-19,1,1,9586,BAKKEN
2,33007016380000,11376,35,5,1500,250,6,6,0,0.0,0,0,1,11370,2010-11-08,1,1,6,DUPEROW
3,33007016380000,11376,47,7,2000,333,6,6,0,0.0,0,0,1,11370,2009-07-29,1,1,6,DUPEROW
4,33007016390000,7942,23,5,1000,250,4,4,0,0.0,0,0,1,7938,2011-08-10,1,1,4,TYLER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6286,33105031490000,20254,207454,21,8713068,895,277,9727,9440,78.4,4056680,417,35,10527,2013-11-01,1,1,9727,BAKKEN
6287,33105031600000,19495,84096,9,3532032,378,311,9330,7120,41.0,3763960,403,30,10165,2013-10-20,1,1,9330,BAKKEN
6288,33105031610000,20425,45872,4,1926624,202,297,9533,7447,25.7,3527746,370,32,10892,2013-10-31,1,1,9533,BAKKEN
6289,33105031620000,20696,68149,7,2862258,314,275,9093,9366,38.6,3667160,403,33,11603,2013-11-06,1,1,9093,BAKKEN


In [10]:
#creating a dataframe with multiple rows per well and a dataframe with no multiples per well
dup=[]
for i in df_nd['api'].unique():
    #print(i)""
    api_rows = df_nd[df_nd['api']==i]
    if len(api_rows)>1:
        #print(api_rows.index.values)
        df_nd.drop(api_rows.index.values, inplace=True)
        dup.append(api_rows)
df_dup=pd.concat(dup)
print('there are ', len(df_nd) ,' well with single entry/row')
#df_dup.to_csv('test.csv')

there are  5594  well with single entry/row


In [11]:
# aggregating multirow data for the same well id into one row
fixed_df=[]
for df_i in dup:
    df_i= df_i.sort_values(['api','top'])
    rows_to_sum=['fluid_bbl','fluid_bbl_per_ft','fluid_gal','fluid_gal_per_ft','propp_lbs','propp_lbs_per_ft']
    rows_to_max=['max_treat_press','max_treat_rate','bottom','treatment_date']
    rows_to_min=['top']
    rows_to_sum_overlap=[]
    rows_to_average=['api']       

                
    if len(df_i['top'].unique())==1:
        if len(df_i['bottom'].unique())==1:
            #one producing interval per well
            assert len(df_i['Produced_Pools'].unique())==1, df_i['Produced_Pools'].unique()
            #treatment by adding more fluids to the same stages
            if len(df_i['stages'].unique())==1:
                assert len(df_i['ft_per_stage'].unique())==1, df_i['ft_per_stage'].unique()
                assert len(df_i['lateral_length'].unique())==1, df_i['lateral_length'].unique()
                #average the same values (=choose one row)
                for ii in ['ft_per_stage','lateral_length', 'stages', 'number_treatments', 'number_intervals','interval']:
                    rows_to_average.append(ii)
            #treatment by adding more stages to the same interval
            else:
                assert len(df_i['lateral_length'].unique())==1, df_i['lateral_length'].unique()
                for ii in ['lateral_length', 'number_intervals','interval']:
                    rows_to_average.append(ii)
                for ii in ['stages', 'number_treatments']:
                    rows_to_sum.append(ii)
                for ii in ['ft_per_stage']:
                    rows_to_min.append(ii)
                #print('muliti',df_i['stages'].unique(), ' ', df_i['ft_per_stage'].unique(), df_i['fluid_bbl'].unique())
        else:
            #lateral is axtended with new stages
            assert len(df_i['Produced_Pools'].unique())==1, df_i['Produced_Pools'].unique()
            for ii in ['number_intervals']:
                rows_to_average.append(ii)
            for ii in ['number_treatments']:
                rows_to_sum.append(ii)
            for ii in ['ft_per_stage']:
                rows_to_min.append(ii)
            for ii in ['lateral_length', 'interval', 'stages']:
                rows_to_max.append(ii)
            #assert len(df_i['lateral_length'].unique())==1, df_i['lateral_length'].unique()
    #multiple treatment intervals
    else:
        assert len(df_i['Produced_Pools'].unique())==1, df_i['Produced_Pools'].unique()
        assert len(df_i['top'].unique())>1, df_i['top'].unique()

        #print([df_i['top'].values[i] for i in range(len(df_i['top']))])
        #print([df_i['bottom'].values[i] for i in range(len(df_i['top']))])

        #Checking for overlapping interval
        overlap=False
        for i in range(len(df_i['top'])-1):
            if df_i['top'].values[i+1]-df_i['bottom'].values[i]<0:
                overlap=True
                df_i['bottom'].values[i]=df_i['bottom'].values[i]+(df_i['top'].values[i+1]-df_i['bottom'].values[i])
                df_i['interval'].values[i]=df_i['bottom'].values[i]-df_i['top'].values[i]
        #print([df_i['bottom'].values[i] for i in range(len(df_i['top']))])

        for ii in ['number_intervals']:
            rows_to_average.append(ii)
        for ii in ['lateral_length', 'stages', 'number_treatments','interval']:
            rows_to_sum.append(ii)
        for ii in ['ft_per_stage']:
            rows_to_min.append(ii)
                #print([df_i['top'].values[i] for i in range(len(df_i['top']))])
                #print([df_i['bottom'].values[i] for i in range(len(df_i['top']))])
                #print(overlap_zone)
        #print(df_i['lateral_length'].values, ' ', df_i['interval'].values, ' ', df_i['stages'].values, ' ', df_i['ft_per_stage'].values, df_i['fluid_bbl'].values)
                
    series_sum=df_i[rows_to_sum].sum()
    series_max=df_i[rows_to_max].max()
    series_min=df_i[rows_to_min].min()
    series_average=df_i[rows_to_average].mean()
    fixed_df.append(series_sum.append([series_max,series_min,series_average]))
fixed=pd.concat(fixed_df, axis=1). transpose()
#fixed.to_csv('test2.csv')
fixed

,fluid_bbl,fluid_bbl_per_ft,fluid_gal,fluid_gal_per_ft,propp_lbs,propp_lbs_per_ft,max_treat_press,max_treat_rate,bottom,treatment_date,top,api,ft_per_stage,lateral_length,stages,number_treatments,number_intervals,interval
0,82,12,3500,583,0,0,0,0,11376,2010-11-08,11370,3.3007e+13,6,6,1,1,1,6
1,80,4,3416,236,0,0,0,0,9417,2010-09-20,9360,3.3007e+13,12,60,4,4,1,45
2,54,0,2276,51,0,0,0,0,9412,2012-08-15,9342,3.3007e+13,31,70,1,2,1,70
3,48466,3,2035572,201,2042220,202,9618,42.9,20525,2011-04-21,10463,3.3007e+13,10062,10062,1,1,1,10062
4,39289,3,1650138,174,2020114,213,8771,41.2,20770,2012-11-30,11327,3.3007e+13,429,9443,30,2,1,9443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,66738,12,2802996,510,1681503,280,9486,36.3,19357,2012-03-19,11920,3.3105e+13,312,11497,36,2,1,7437
248,73233,17,3075786,753,3939391,784,9341,36.7,21123,2012-06-01,11040,3.3105e+13,279,10083,34,2,1,10083
249,52566,5,2207772,251,3103049,352,8000,37.7,19260,2012-06-22,10260,3.3105e+13,313,18297,29,2,1,9000
250,89045,21,3739890,911,3884477,906,9906,31.3,20761,2013-08-05,11762,3.3105e+13,286,8700,28,2,1,8700


In [12]:
#fixing data types
fixed=fixed.astype('int64')
fixed['max_treat_rate']=fixed['max_treat_rate'].astype('float64')
fixed['treatment_date']=fixed['treatment_date'].astype('datetime64[ns]')
fixed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fluid_bbl          252 non-null    int64         
 1   fluid_bbl_per_ft   252 non-null    int64         
 2   fluid_gal          252 non-null    int64         
 3   fluid_gal_per_ft   252 non-null    int64         
 4   propp_lbs          252 non-null    int64         
 5   propp_lbs_per_ft   252 non-null    int64         
 6   max_treat_press    252 non-null    int64         
 7   max_treat_rate     252 non-null    float64       
 8   bottom             252 non-null    int64         
 9   treatment_date     252 non-null    datetime64[ns]
 10  top                252 non-null    int64         
 11  api                252 non-null    int64         
 12  ft_per_stage       252 non-null    int64         
 13  lateral_length     252 non-null    int64         
 14  stages    

In [13]:
# merging fixed multirow data back to single row dataset
df_clean = pd.concat([df_nd,fixed])
del df_clean['Produced_Pools']
del df_clean['number_intervals']
print(len(df_nd), ' + ', len(fixed), ' = ', len(df_nd)+len(fixed))
df_clean.info()

5594  +  252  =  5846
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5846 entries, 0 to 251
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   api                5846 non-null   int64         
 1   bottom             5846 non-null   int64         
 2   fluid_bbl          5846 non-null   int64         
 3   fluid_bbl_per_ft   5846 non-null   int64         
 4   fluid_gal          5846 non-null   int64         
 5   fluid_gal_per_ft   5846 non-null   int64         
 6   ft_per_stage       5846 non-null   int64         
 7   lateral_length     5846 non-null   int64         
 8   max_treat_press    5846 non-null   int64         
 9   max_treat_rate     5846 non-null   float64       
 10  propp_lbs          5846 non-null   int64         
 11  propp_lbs_per_ft   5846 non-null   int64         
 12  stages             5846 non-null   int64         
 13  top                5846 non-null   int64  

In [14]:
#merge dataset together (now each well has only one row)
df_clean_compl = pd.merge(df_clean, df_label,on=['api'])
df_clean_compl = pd.merge(df_clean_compl, df_wells,on=['api'])
df_clean_compl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5846 entries, 0 to 5845
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   api                 5846 non-null   int64         
 1   bottom              5846 non-null   int64         
 2   fluid_bbl           5846 non-null   int64         
 3   fluid_bbl_per_ft    5846 non-null   int64         
 4   fluid_gal           5846 non-null   int64         
 5   fluid_gal_per_ft    5846 non-null   int64         
 6   ft_per_stage        5846 non-null   int64         
 7   lateral_length      5846 non-null   int64         
 8   max_treat_press     5846 non-null   int64         
 9   max_treat_rate      5846 non-null   float64       
 10  propp_lbs           5846 non-null   int64         
 11  propp_lbs_per_ft    5846 non-null   int64         
 12  stages              5846 non-null   int64         
 13  top                 5846 non-null   int64       

In [15]:
#saving cleaned dataset to csv file
df_clean_compl.to_csv('DataWrangling_clean_dataset.csv')